## Install package and download required data files

In [ ]:
# Download ADSO ontology file
!wget "https://raw.githubusercontent.com/viraj-lakshitha/animal-disease-symptom-ontology/develop/ADSOv1.0.3.owl"

In [7]:
!pip install rdflib spacy sentence-transformers huggingface-hub transformers

## Load required pre-trained models




In [8]:
# Download Wordnet dependencies
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
from sentence_transformers import SentenceTransformer, util
from huggingface_hub import from_pretrained_keras
import numpy as np
import transformers
import tensorflow as tf

# Text similarity model
tsm = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2');

# Semantic similarity model
ssm = from_pretrained_keras("keras-io/bert-semantic-similarity")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/6.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/30.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.88M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

## Helper Utils

This block contains functions/classes/enums that use throught out the entity disambiguation component.

In [93]:
# Help utils for semantic similarity model
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data."""
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=32,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

In [92]:
labels = ["negative_similarity", "positive_similarity", "neutral"]

## Data loader component

In here we are going to retreive the all type of entities/nodes from the ontology knowledgebase and store in seperate arrays.



In [77]:
# Load downloaded ontology graph
from rdflib import Graph, Namespace, Literal, RDF, URIRef

g = Graph()
g.parse("/content/ADSOv1.0.3.owl")

<Graph identifier=Ne16218f383354f72b39dd935e4377809 (<class 'rdflib.graph.Graph'>)>

In [78]:
# Get the URIRef for given keyword
def get_ref(keyword):
    return URIRef("https://ontology.drpawspaw.com/"+keyword)

In [79]:
# Get the entity 'text' from the URIRef
def get_text_from_uri(uri):
    for s,p,o in g:
        if s == uri and p == URIRef("https://ontology.drpawspaw.com/text"):
            return o

In [80]:
# Collect named entities from ontology
diseases = []
symptoms = []
synonyms = []

for s,p,o in g:
    if p == get_ref("hasDisease"):
        try:
            dis = get_text_from_uri(o).toPython()
            diseases.append(dis)
        except:
            print(type(o), o)
    if p == get_ref("hasSymptom"):
        try:
            sym = get_text_from_uri(o).toPython()
            symptoms.append(sym)
        except:
            print(type(o), o)
    if p == get_ref("hasSynonym"):
        try:
            syn = get_text_from_uri(o).toPython()
            synonyms.append(syn)
        except:
            print(type(o), o)

<class 'rdflib.term.URIRef'> https://ontology.drpawspaw.com/ADSO0000000110


In [81]:
# List of symptoms and their synonyms
symp_syn = []

# Check the keyword already exist or not in collection
def is_exist_ss(keyword):
    for _,k,_ in symp_syn:
        if k == keyword:
            return True
            
# Apped all the symptoms and their synonyms
for s,p,o in g: # subject, predicate, object
    # filter synonyms
    if p == get_ref("hasSymptom"):
        try:
            x = get_text_from_uri(o).toPython()
        except:
            continue
        curr_symp_syn = []
        for s1, p1, o1 in g:
            # filter synonym for above "o" entity
            if s1 == o and p1 == get_ref("hasSynonym"):
                try:
                    y = get_text_from_uri(o1).toPython()
                    curr_symp_syn.append(y)                    
                except:
                    continue
        # validate to add only one entry
        if not is_exist_ss(x):
            try:
                idx = o.toPython()
                symp_syn.append((idx, x, curr_symp_syn))
            except:
                continue

In [82]:
symp_syn[:5]

[('https://ontology.drpawspaw.com/ADSO0000000098',
  'Odor',
  ['Unpleasant Smell', 'Bad Smell']),
 ('https://ontology.drpawspaw.com/ADSO0000000039', 'Paralysis', []),
 ('https://ontology.drpawspaw.com/ADSO0000000033', 'Skin Sores', []),
 ('https://ontology.drpawspaw.com/ADSO0000000065', 'Unusual Collapse', []),
 ('https://ontology.drpawspaw.com/ADSO0000000022', 'Rapid Heart Beat', [])]

In [83]:
len(symp_syn)

79

## Text similarity

In here we are going to calculate the similarity of the words using pretrained model from Hugging Face => Text Similarity - To get the similar words for identified named entities (https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2, https://huggingface.co/tasks/sentence-similarity)

In [76]:
# Generate synonyms for the existing symptoms and update the list of synonyms
from nltk.corpus import wordnet

def generate_synonyms(uri, keyword, syns):
    for synonym in wordnet.synsets(keyword):
        for item in synonym.lemmas():
            if keyword != synonym.name() and len(synonym.lemma_names()) > 1:
                syns.append(item.name())
    return (uri, keyword, syns)

In [41]:
# for idx,(uri,keyword,syns) in enumerate(symp_syn):
#   symp_syn[idx] = generate_synonyms(uri,keyword,syns)

In [86]:
print(len(symp_syn))
print(symp_syn[:5])

79
[('https://ontology.drpawspaw.com/ADSO0000000098', 'Odor', ['Unpleasant Smell', 'Bad Smell']), ('https://ontology.drpawspaw.com/ADSO0000000039', 'Paralysis', []), ('https://ontology.drpawspaw.com/ADSO0000000033', 'Skin Sores', []), ('https://ontology.drpawspaw.com/ADSO0000000065', 'Unusual Collapse', []), ('https://ontology.drpawspaw.com/ADSO0000000022', 'Rapid Heart Beat', [])]


In [87]:
# Exapand the all entities
expanded_symp = []

def is_exist_es(keyword):
    for idx,word in expanded_symp:
        if keyword == word:
            return True

for idx,word,syns in symp_syn:
    for s in syns:
        if not is_exist_es(s):
            expanded_symp.append((idx, s))
    if not is_exist_es(word):
        expanded_symp.append((idx, word))

In [88]:
print(len(expanded_symp))
print(expanded_symp[:5])

101
[('https://ontology.drpawspaw.com/ADSO0000000098', 'Unpleasant Smell'), ('https://ontology.drpawspaw.com/ADSO0000000098', 'Bad Smell'), ('https://ontology.drpawspaw.com/ADSO0000000098', 'Odor'), ('https://ontology.drpawspaw.com/ADSO0000000039', 'Paralysis'), ('https://ontology.drpawspaw.com/ADSO0000000033', 'Skin Sores')]


In [89]:
# Calculate the similariity
# Here "tsm" means the text-similarity-model, that we loaded in "Load required pre-trained models"
# "get_similarity" accepts the URI, identified named entity, entity from ontology
def get_similarity(idx, word, ne): # Return a tuple, contain URI, node, similarity_score
    embedding_1 = tsm.encode(ne, convert_to_tensor=True)
    embedding_2 = tsm.encode(word, convert_to_tensor=True)
    return (idx, word, util.pytorch_cos_sim(embedding_1, embedding_2))

In [90]:
# At the current implementation, we only get first five entities
# "entity" - identified named entity, "nodes" - named entities in the onotology (in format of [(idx, text)])
def get_most_similarity_entities(entity, nodes):
  scores = map(lambda e: get_similarity(e[0], e[1], entity), nodes)
  return sorted(list(scores), key=lambda x:x[2], reverse=True)[:5]

In [91]:
# Tryout the implementation
identified_ne = ["fever", "lethargy"]

# Get the top five entities from the text similarity, accroding to the cosine similarity score
get_most_similarity_entities(identified_ne[0], expanded_symp)

[('https://ontology.drpawspaw.com/ADSO0000000005', 'Fever', tensor([[1.]])),
 ('https://ontology.drpawspaw.com/ADSO0000000041',
  'High Fever',
  tensor([[0.8284]])),
 ('https://ontology.drpawspaw.com/ADSO0000000032',
  'Pneumonia',
  tensor([[0.4948]])),
 ('https://ontology.drpawspaw.com/ADSO0000000041',
  'High Temperature',
  tensor([[0.4413]])),
 ('https://ontology.drpawspaw.com/ADSO0000000008',
  'Vomiting',
  tensor([[0.4372]]))]

## Semantic Similarity

In here we are going to calculate the semantic similarity of the words using pretrained model from HuggingFace => Semantic Similarity - To get the similarity in context for the ranked candidates (https://huggingface.co/keras-io/bert-semantic-similarity)

In [99]:
# Calculate the semantic similarity of the identified entity and nodes in ontology
# "calculate_semantic_similarity" functions accpets, the URI, text and identified entity
def calculate_semantic_similarity(uri, ctx, entity):
    sentence_pairs = np.array([[str(entity), str(ctx)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    probs = ssm.predict(test_data[0])[0]
    labels_probs = {labels[i]: float(probs[i]) for i, _ in enumerate(labels)}
    return (uri, ctx, labels_probs["positive_similarity"])

In [ ]:
# "get_highest_ctx_similarity" function return the highest context similarity as tuple (URI, Word, ctx_similarity)
def get_highest_ctx_similarity(entity, nodes)):
  scores = map(lambda e: calculate_semantic_similarity(e[0], e[1], entity), nodes)
  return sorted(list(scores), key=, reverse=True)[:1]

In [ ]:
ine = identified_ne[0]
candidates = get_most_similarity_entities(ine, expanded_symp) # Candidates from text-similarity
get_highest_ctx_similarity(ine, candidates)